In [30]:
import pandas as pd
import numpy as np
import nltk,re
import gensim
import os

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional, Embedding, Dropout, Activation, Flatten,
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.initializers import Constant 

from nltk import FreqDist
nltk.download('stopwords')



SyntaxError: trailing comma not allowed without surrounding parentheses (<ipython-input-30-18406d796481>, line 13)

In [4]:
def read_from_file(file_name):
    X = []
    with open(file_name, encoding="utf8") as f:             #StackOverflow.txt
        for x in f:
            X.append(x)
    return X

In [5]:
train_data_X = pd.Series(read_from_file('StackOverflow.txt'))
train_data_Y = pd.Series(read_from_file('StackOverflow_gnd.txt'))

In [6]:
def clean_text(text):
    text = text.split()
    sentence_list  = list()
    stop_words = set(stopwords.words("english"))
    text = [w for w in text if w not in stop_words]
    
    text = " ".join(text)
    
    text = text.lower()
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", "  ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", "  ", text)
    text = re.sub(r"\:", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    lemmatize_words = [lemmatizer.lemmatize(word) for word in text]  
    text = " ".join(lemmatize_words)
    
    sentence_list.append(text)
    return text

In [7]:
train_data_X = train_data_X.apply(lambda x: clean_text(x))
train_data_Y = train_data_Y.apply(lambda x: clean_text(x))

In [8]:
print(train_data_X)
print(train_data_Y)


0        how i fill dataset datatable linq query resultset
1                                 how page collection linq
2                best subversion client window vista 64bit
3        best practice collaborative environment bin di...
4        visual studio setup project per user registry ...
5        how i elegantly express left join aggregate sq...
6                        net xml comment api documentation
7        modify address bar url ajax app match current ...
8        integrating visual studio test project cruise ...
9        what longtime window user know starting use linux
10                   folder project visual studio solution
11                                 how i create branch svn
12                 add custom tag visual studio validation
13         how i turn line number default textwrangler mac
14               how tab focus onto dropdown field mac osx
15                           how tab button osx dialog box
16                                 progressive enhanceme

In [9]:
sen_len=[]
word_list = []
for text in train_data_X:
    word=word_tokenize(text)
    l=len(word)
    sen_len.append(l)
    word_list.append(word)
    
max_sen_len=np.max(sen_len)
print(word_list)


[['how', 'i', 'fill', 'dataset', 'datatable', 'linq', 'query', 'resultset'], ['how', 'page', 'collection', 'linq'], ['best', 'subversion', 'client', 'window', 'vista', '64bit'], ['best', 'practice', 'collaborative', 'environment', 'bin', 'directory', 'svn'], ['visual', 'studio', 'setup', 'project', 'per', 'user', 'registry', 'setting'], ['how', 'i', 'elegantly', 'express', 'left', 'join', 'aggregate', 'sql', 'linq', 'query'], ['net', 'xml', 'comment', 'api', 'documentation'], ['modify', 'address', 'bar', 'url', 'ajax', 'app', 'match', 'current', 'state'], ['integrating', 'visual', 'studio', 'test', 'project', 'cruise', 'control'], ['what', 'longtime', 'window', 'user', 'know', 'starting', 'use', 'linux'], ['folder', 'project', 'visual', 'studio', 'solution'], ['how', 'i', 'create', 'branch', 'svn'], ['add', 'custom', 'tag', 'visual', 'studio', 'validation'], ['how', 'i', 'turn', 'line', 'number', 'default', 'textwrangler', 'mac'], ['how', 'tab', 'focus', 'onto', 'dropdown', 'field', 'm

In [10]:
#Tagging each word with its parts of speech
pos_value_sg = []
word_value_sg = []

for i in range (0,len(word_list)):
    pos_tag = nltk.pos_tag(word_list[i])
    word_key,pos_value = zip(*pos_tag)
    word_value_sg.append(word_key)
    pos_value_sg.append(pos_value)
    print(word_key,pos_value)


('how', 'i', 'fill', 'dataset', 'datatable', 'linq', 'query', 'resultset') ('WRB', 'JJ', 'VBP', 'VBN', 'JJ', 'JJ', 'NN', 'NN')
('how', 'page', 'collection', 'linq') ('WRB', 'JJ', 'NN', 'NN')
('best', 'subversion', 'client', 'window', 'vista', '64bit') ('JJS', 'NN', 'NN', 'NN', 'NN', 'CD')
('best', 'practice', 'collaborative', 'environment', 'bin', 'directory', 'svn') ('JJS', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN')
('visual', 'studio', 'setup', 'project', 'per', 'user', 'registry', 'setting') ('JJ', 'NN', 'NN', 'NN', 'IN', 'JJ', 'NN', 'VBG')
('how', 'i', 'elegantly', 'express', 'left', 'join', 'aggregate', 'sql', 'linq', 'query') ('WRB', 'JJ', 'RB', 'VBP', 'VBN', 'NN', 'JJ', 'JJ', 'NN', 'NN')
('net', 'xml', 'comment', 'api', 'documentation') ('JJ', 'JJ', 'NN', 'NN', 'NN')
('modify', 'address', 'bar', 'url', 'ajax', 'app', 'match', 'current', 'state') ('NN', 'NN', 'NN', 'JJ', 'JJ', 'NN', 'NN', 'JJ', 'NN')
('integrating', 'visual', 'studio', 'test', 'project', 'cruise', 'control') ('VBG', 'JJ

('changing', 'ctrl', 'tab', 'behavior', 'moving', 'document', 'visual', 'studio') ('VBG', 'NN', 'NN', 'NN', 'VBG', 'JJ', 'JJ', 'NN')
('how', 'i', 'use', 'linq', 'paging', 'generic', 'collection') ('WRB', 'JJ', 'VBP', 'JJ', 'VBG', 'JJ', 'NN')
('mocking', 'iqueriable', '<', 't') ('VBG', 'JJ', 'NN', 'NN')
('source', 'control', 'visual', 'studio', 'isolated', 'shell') ('NN', 'NN', 'JJ', 'NN', 'VBD', 'NN')
('accessing', 'excel', 'data', 'source', 'running', 'ssis', 'package', '64', 'bit', 'server') ('VBG', 'NN', 'NNS', 'NN', 'VBG', 'NN', 'NN', 'CD', 'NN', 'NN')
('intellisenss', 'xelements', 'object', 'xml', 'schema') ('NN', 'NNS', 'VBP', 'CD', 'NN')
('transactional', 'design', 'pattern') ('JJ', 'NN', 'NN')
('sharepoint', 'comexception', '0x81020037') ('NN', 'NN', 'CD')
('getting', 'error', 'manage', 'category', 'found', 'attribute', 'navigation', 'column') ('VBG', 'NN', 'NN', 'NN', 'VBD', 'JJ', 'NN', 'NN')
('virtual', 'mac') ('JJ', 'NN')
('how', 'i', 'find', 'excel', 'column', 'name', 'corr

('visual', 'studio', '2008', 'randomly', 'hang', 'test', 'run') ('JJ', 'NN', 'CD', 'NN', 'VBD', 'NN', 'NN')
('wordpress', 'term', 'exist', 'desired', 'effect') ('JJ', 'NN', 'NN', 'VBD', 'NN')
('drupal', 'rule', 'flagging', 'multiple', 'term', 'user', 'save') ('JJ', 'NN', 'VBG', 'JJ', 'NN', 'NN', 'VB')
('excel', 'vba', 'get', 'hwnd', 'value', 'commandbutton') ('NN', 'NN', 'VBP', 'JJ', 'NN', 'NN')
('visual', 'studio', '2005', 'crash', 'start', 'up') ('JJ', 'NN', 'CD', 'NN', 'VBD', 'RP')
('what', 'version', 'tinymce', 'work', 'drupal', '5', 'google', 'chrome') ('WP', 'NN', 'NN', 'NN', 'JJ', 'CD', 'NN', 'NN')
('what', 'simplest', 'way', 'write', 'web', 'apps', 'haskell') ('WP', 'JJS', 'NN', 'JJ', 'NN', 'NN', 'NN')
('modifying', 'spreadsheet', 'using', 'vb', 'macro') ('VBG', 'NN', 'VBG', 'NNS', 'NN')
('how', 'i', 'stop', 'visual', 'studio', 'automatically', 'inserting', 'asterisk', 'block', 'comment') ('WRB', 'JJ', 'VBP', 'JJ', 'NN', 'RB', 'VBG', 'JJ', 'NN', 'NN')
('can', 'stp', 'template',

('is', 'safe', 'manipulate', 'object', 'created', 'outside', 'thread', 'explicitly', 'access', 'thread', 'created', 'them') ('VBZ', 'JJ', 'NN', 'NN', 'VBD', 'JJ', 'NN', 'RB', 'NN', 'NN', 'VBD', 'PRP')
('releasing', 'license', 'do', 'file', 'need', 'copyright', 'statment') ('VBG', 'NN', 'VBP', 'RB', 'VB', 'JJ', 'NN')
('how', 'prevent', 'printing', 'dialog', 'using', 'excel', 'printout', 'method') ('WRB', 'JJ', 'NN', 'NN', 'VBG', 'JJ', 'NN', 'NN')
('why', 'team', 'foundation', 'server', 'express', 'edition') ('WRB', 'NN', 'NN', 'NN', 'NN', 'NN')
('good', 'way', 'learn', 'cocoa') ('JJ', 'NN', 'VBP', 'NN')
('what', 'favorite', 'little', 'known', 'command', 'line', 'trick', 'using', 'bash') ('WP', 'JJ', 'JJ', 'VBN', 'NN', 'NN', 'NN', 'VBG', 'NN')
('how', 'merge', 'branch', 'branch', 'back', 'bidirectional', 'merging', 'svn') ('WRB', 'NN', 'NN', 'NN', 'RB', 'JJ', 'NN', 'NN')
('what', 'ide', 'editor', 'choice', 'haskell') ('WP', 'NN', 'NN', 'NN', 'NN')
('how', 'stop', 'visual', 'studio', 'gen

('osx', 'security', 'framework', 'nameandpassword', 'sample', 'application') ('JJ', 'NN', 'NN', 'NN', 'NN', 'NN')
('what', 'difference', 'view', 'materialized', 'view', 'oracle') ('WP', 'NN', 'NN', 'VBN', 'NN', 'NN')
('moving', 'v', 'svn') ('VBG', 'NN', 'NN')
('number', 'class', 'visual', 'studio', 'solution') ('NN', 'NN', 'JJ', 'NN', 'NN')
('inconsistent', 'display', 'behavior', 'quick', 'launch', 'menu', 'moss', '2007') ('JJ', 'NN', 'NN', 'JJ', 'NN', 'NN', 'NN', 'CD')
('why', 'statsvn', 'fail', 'claiming', 'directory', 'working', 'copy') ('WRB', 'JJ', 'NN', 'VBG', 'NN', 'VBG', 'NN')
('how', 'switch', 'back', 'previous', 'version', 'file', 'without', 'deleting', 'subsequent', 'revision') ('WRB', 'VB', 'RB', 'JJ', 'NN', 'NN', 'IN', 'VBG', 'JJ', 'NN')
('learning', 'drupal', 'fast', 'track', 'create', 'stackoverflow', 'clone') ('VBG', 'JJ', 'JJ', 'NN', 'NN', 'NN', 'NN')
('how', 'i', 'automate', 'process', 'deploying', 'infopath', 'form', 'sharepoint', 'server', '2007') ('WRB', 'JJ', 'VBP

...

('magento', 'call', 'variable', 'customer', 's', 'shipping', 'address', 'onepage', 'checkout', 'step', '3', 'shipping', 'method') ('NN', 'NN', 'JJ', 'NN', 'NN', 'VBG', 'JJ', 'NN', 'JJ', 'NN', 'CD', 'NN', 'NN')
('write', 'identity', 'function', 'haskell') ('JJ', 'NN', 'NN', 'NN')
('can', 'i', 'something', 'like', 'haskell') ('MD', 'VB', 'NN', 'IN', 'NN')
('constraining', 'returntype', 'context') ('VBG', 'NN', 'NN')
('ghc', 'rts', 'option', 'garbage', 'collection') ('NN', 'NNS', 'NN', 'NN', 'NN')
('haskell', 'function', 'sometimes', 'return', 'function') ('NN', 'NN', 'RB', 'JJ', 'NN')
('proper', 'way', 'import', 'thing', 'already', 'defined', 'prelude', 'haskell') ('JJ', 'NN', 'JJ', 'NN', 'RB', 'VBN', 'NN', 'NN')
('haskell', 'map', 'zip', 'v', 'list', 'comprehension') ('NN', 'NN', 'NN', 'NN', 'NN', 'NN')
('wxhaskell', 'asynchronous', 'update') ('RB', 'JJ', 'NN')
('after', 'overriding', 'contoller', 'best', 'update', 'handle') ('IN', 'VBG', 'NN', 'JJS', 'NN', 'NN')
('haskell', 'sorting', 

('why', 'following', 'simple', 'haskell', 'function', 'give', 'slightly', 'strange', 'answer') ('WRB', 'VBG', 'NN', 'NN', 'NN', 'VBP', 'RB', 'JJ', 'NN')
('product', 'insert', 'funtion', 'can', 'not', 'find', 'function', 'insert', 'product', 'database') ('NN', 'JJ', 'NN', 'MD', 'RB', 'VB', 'NN', 'JJ', 'NN', 'NN')
('inserted', 'custom', 'option', 'displaying', 'display', 'refresh') ('JJ', 'NN', 'NN', 'VBG', 'NN', 'NN')
('use', 'type', 'synonym', 'monad', 'transformer') ('NN', 'NN', 'NN', 'NN', 'NN')
('stripping', 'newlines', 'haskell') ('VBG', 'NNS', 'NN')
('state', 'monad', 'haskell') ('NN', 'NNS', 'VBP')
('operating', 'return', 'maybe', 'contains', 'just') ('VBG', 'NN', 'RB', 'VBZ', 'RB')
('magento', 'xml', 'action', 'path') ('NN', 'JJ', 'NN', 'NN')
('tip', 'elegant', 'code', 'monad') ('NN', 'JJ', 'NN', 'NN')
('renaming', 'current', 'module') ('VBG', 'JJ', 'NN')
('haskell', 'nonobvious', 'example', 'functional', 'dependency') ('RB', 'JJ', 'NN', 'JJ', 'NN')
('memoization', 'project', 'e

('magento', 'image', 'upload', 'problem') ('NN', 'NN', 'NN', 'NN')
('configure', 'product', 'like') ('NN', 'NN', 'IN')
('adjusting', 'order', 'quantity', 'magento') ('VBG', 'NN', 'NN', 'NN')
('magento', 'getting', 'attribute', 'nav', 'filter', 'item') ('IN', 'VBG', 'JJ', 'JJ', 'NN', 'NN')
('is', 'possible', 'customize', 'magento', 'admin', 'area') ('VBZ', 'JJ', 'NN', 'NN', 'JJ', 'NN')
('how', 'get', 'data', 'entity', 'for', 'example', 'customer', 'eav', 'attribute', 'table', 'shown', 'customer', 'grid', 'admin') ('WRB', 'VB', 'NNS', 'NN', 'IN', 'NN', 'NN', 'IN', 'NN', 'NN', 'VBN', 'NN', 'NN', 'NN')
('magento', 'show', 'review', 'step', 'one', 'page', 'checkout') ('NN', 'NN', 'JJ', 'NN', 'CD', 'NN', 'NN')
('magento', 'case', 'tiered', 'pricing', 'able', 'break', 'case', 'price', 'individual', 'skus') ('NN', 'NN', 'VBD', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN', 'NN')
('showing', 'magento', 'new', 'product', 'like', 'standard', 'category') ('VBG', 'JJ', 'JJ', 'NN', 'IN', 'JJ', 'NN')
('event', 

('magento', 'indexcontrollor', 'php', 'url', 'rewriting', 'working', 'properly') ('NN', 'NN', 'NN', 'JJ', 'VBG', 'VBG', 'RB')
('url', 'cm', 'page', 'magento') ('JJ', 'NN', 'NN', 'NN')
('2', 'customer', 'created', 'master', 'slave', 'configured') ('CD', 'NN', 'VBD', 'NN', 'NN', 'VBD')
('what', 'block', 'type', 'left', 'column', 'magento', 'theme') ('WP', 'NN', 'NN', 'VBD', 'NN', 'NN', 'NN')
('display', 'new', 'arrival', 'category', 'magento') ('VB', 'JJ', 'JJ', 'NN', 'NN')
('invoice', '2', 'order', 'magento') ('RB', 'CD', 'NN', 'NN')
('restrict', 'user', 'view', 'product', 'magento') ('VB', 'JJ', 'NN', 'NN', 'NN')
('magento', 'checking', 'module', 'installed') ('NN', 'VBG', 'NN', 'VBD')
('write', 'custom', 'panel', 'displaying', 'category', 'magento') ('JJ', 'NN', 'NN', 'VBG', 'NN', 'NN')
('magento', '1', '4', 'configurable', 'product', 'sku') ('NN', 'CD', 'CD', 'JJ', 'NN', 'NN')
('using', 'catalog', 'image', 'full', 'background', 'image', 'magento') ('VBG', 'NN', 'NN', 'JJ', 'NN', 'NN'

('access', 'filter', 'var', 'attribute') ('NN', 'NN', 'NN', 'NN')
('add', 'cart', 'link', 'with', 'number', 'item', 'sidebar', 'magento') ('VB', 'JJ', 'NN', 'IN', 'NN', 'NN', 'NN', 'NN')
('error', 'adding', 'attribute', 'magento', '1', '4', '2') ('NN', 'VBG', 'JJ', 'NN', 'CD', 'CD', 'CD')
('issue', 'advance', 'featured', 'product', 'extension', 'magento', '1', '4', '2') ('NN', 'NN', 'VBD', 'NN', 'NN', 'VBD', 'CD', 'CD', 'CD')
('how', 'change', 'product', 'display', 'order', 'magento') ('WRB', 'JJ', 'NN', 'NN', 'NN', 'NN')
('magento', 'multiselect', 'color', 'attribute') ('NN', 'NN', 'NN', 'NN')
('magento', 'onepage', 'issue') ('NN', 'NN', 'NN')
('magento', 'create', 'helper', 'class') ('NN', 'NN', 'NN', 'NN')
('get', 'order', 'id', 'magento') ('VB', 'NN', 'NN', 'NN')
('magento', 'adminhtml', 'default', 'website', 'new', 'customer', 'form') ('NN', 'NN', 'NN', 'JJ', 'JJ', 'NN', 'NN')
('magento', 'determine', 'id', 'order', 'placed', 'admin') ('NNS', 'VBP', 'JJ', 'NN', 'VBD', 'NN')
('weir

In [11]:
def get_no_unique_words(texts_1):
    all_words = ' '.join(texts_1)
    all_words = word_tokenize(all_words)
    dist = FreqDist(all_words)
    num_unique_word = len(dist)
    return num_unique_word

In [12]:
num_unique_word = get_no_unique_words(train_data_X)
num_unique_word

10033

In [13]:
EMBEDDING_DIM = 32

model = gensim.models.Word2Vec(sentences = word_list, size = EMBEDDING_DIM, min_count=1,workers=4,window=5)

words = list(model.wv.vocab)

print('Vocabulary size %d' %len(words))

Vocabulary size 10033


In [14]:
print(model.wv.most_similar('exceeds'))

[('pointcut', 0.9252736568450928), ('bill', 0.9234253168106079), ('pia', 0.9218239188194275), ('intended', 0.9204587340354919), ('toggling', 0.9200159907341003), ('who', 0.9185937643051147), ('subregex', 0.918592631816864), ('nsdateformatter', 0.917363703250885), ('upper', 0.917073130607605), ('german', 0.9162404537200928)]


In [15]:
filename = 'word_embedding.txt'
model.wv.save_word2vec_format(filename, binary= False)

In [16]:
embedding_index = {}
f = open(os.path.join('','word_embedding.txt'),encoding = 'utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coeffs = np.asarray(values[1:])
    embedding_index[word] = coeffs
f.close()    

In [17]:
tokenizer = Tokenizer(num_words=10033)
tokenizer.fit_on_texts(word_list)

train_data_X_sequences = tokenizer.texts_to_sequences(word_list)

word_index = tokenizer.word_index
print(word_index)

{'how': 1, 'i': 2, 'file': 3, 'using': 4, 'hibernate': 5, 'excel': 6, 'wordpress': 7, 'magento': 8, 'drupal': 9, 'linq': 10, 'spring': 11, 'matlab': 12, 'scala': 13, 'oracle': 14, 'ajax': 15, 'sharepoint': 16, 'what': 17, 'visual': 18, 'haskell': 19, 'studio': 20, 'bash': 21, 'is': 22, 'apache': 23, 'svn': 24, 'qt': 25, 'get': 26, 'use': 27, 'list': 28, 'way': 29, 'page': 30, 'error': 31, 'mac': 32, 'function': 33, 'type': 34, 'data': 35, 'can': 36, 'problem': 37, 'query': 38, 'sql': 39, 'value': 40, 'application': 41, 'x': 42, 'custom': 43, 'add': 44, 'object': 45, 'script': 46, 'o': 47, 'multiple': 48, 'web': 49, 'c': 50, 'best': 51, 'user': 52, 'server': 53, 'why': 54, 'cocoa': 55, 'code': 56, 'create': 57, 'window': 58, 'one': 59, 'view': 60, 'table': 61, 'class': 62, 'net': 63, 'subversion': 64, 'change': 65, 'post': 66, 'string': 67, 'make': 68, 'in': 69, '2': 70, 'not': 71, 'set': 72, 'field': 73, 'form': 74, 'project': 75, 'v': 76, 'without': 77, 'image': 78, 'php': 79, 'xml': 

In [18]:
print(train_data_X[0:10])
print(train_data_X_sequences[0:10])

print(word_list[0:10])


0    how i fill dataset datatable linq query resultset
1                             how page collection linq
2            best subversion client window vista 64bit
3    best practice collaborative environment bin di...
4    visual studio setup project per user registry ...
5    how i elegantly express left join aggregate sq...
6                    net xml comment api documentation
7    modify address bar url ajax app match current ...
8    integrating visual studio test project cruise ...
9    what longtime window user know starting use linux
dtype: object
[[1, 2, 1179, 971, 1180, 10, 38, 1504], [1, 30, 138, 10], [51, 64, 241, 58, 1308, 1863], [51, 280, 2572, 330, 1246, 91, 24], [18, 20, 409, 75, 451, 52, 2573, 163], [1, 2, 2981, 709, 452, 182, 1309, 39, 10, 38], [63, 80, 343, 217, 871], [562, 479, 520, 86, 15, 113, 344, 264, 352], [897, 18, 20, 188, 75, 2982, 110], [17, 4962, 58, 52, 414, 838, 27, 300]]
[['how', 'i', 'fill', 'dataset', 'datatable', 'linq', 'query', 'resultset'], ['ho

In [19]:
train_data_X_sequences = pad_sequences(train_data_X_sequences, maxlen = max_sen_len)

#labels = np.array(labels)

In [20]:
print(train_data_X_sequences[0:10])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    1    2 1179  971 1180   10   38 1504]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    1   30  138   10]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   51   64  241   58 1308 1863]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   51  280 2572  330 1246   91   24]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   18   20  409   75  451   52 2573  163]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    1    2 2981  709  452  182 1309   39   10   38]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0   63   80  

In [21]:
num_words = len(word_index) + 1

embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [22]:
print(num_words)

10034


In [23]:
#print(train_data_X_sequences.shape)
print(train_data_Y.shape)
train_data_Y = to_categorical(train_data_Y)
print(train_data_Y.shape)

(20000,)
(20000, 21)


In [35]:
model1=Sequential()
model1.add(Embedding(num_words,EMBEDDING_DIM,embeddings_initializer= Constant(embedding_matrix),input_length = 28,
                     mask_zero=True, trainable = True))
model1.add(Bidirectional(LSTM(64,return_sequences=True)))
#model1.add(LSTM(64,return_sequences=True,go_backwards=True))
model1.add(Dense(21,activation='softmax'))
model_compiled = model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 28, 32)            321088    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 28, 128)           49664     
_________________________________________________________________
dense_3 (Dense)              (None, 28, 21)            2709      
Total params: 373,461
Trainable params: 373,461
Non-trainable params: 0
_________________________________________________________________


In [36]:
batch_size = 128
epochs = 3
num_classes=21
history1=model1.fit(train_data_X_sequences, train_data_Y,epochs=epochs, batch_size=batch_size, verbose=1)



ValueError: Error when checking target: expected dense_3 to have 3 dimensions, but got array with shape (20000, 21)

In [26]:
target_layer = model1.layers[-2]
target_layer.return_sequences = True
target_layer.forward_layer.return_sequences = True
target_layer.backward_layer.return_sequences = True

In [33]:
outputs = target_layer.get_input_at(target_layer.input)
m = Model(model1.input, outputs)

AttributeError: Layer bidirectional_2 has multiple inbound nodes, hence the notion of "layer input" is ill-defined. Use `get_input_at(node_index)` instead.